In [ ]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path))

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
import vectorian

from vectorian.importers import NovelImporter
from vectorian.embeddings import FastText
from vectorian.session import LabSession

embedding = FastText("en")

im = NovelImporter(nlp)
doc = im("/Users/arbeit/A Child's Dream of a Star.txt")

session = LabSession(
    [doc],
    [embedding])

Matching checksum for /Users/arbeit/Projects/vectorian-2021/vectorian/core/cpp/core.cpp --> not compiling


In [4]:
query = nlp("literal")
r = session.find(query, n=3)

FloatProgress(value=0.0, layout=Layout(width='100%'), max=1.0)

In [5]:
r

KeyError: '\n\t\t\t        \tf'